This notebook contains a simple spam classifier trained on the SMS Spam Collection Dataset (data source: https://www.kaggle.com/uciml/sms-spam-collection-dataset).

In this version, a pre-trained BERT model is adopted and fine-tuned on the SMS texts corpus. A tensorflow_hub version of BERT - easily accessible as a Keras layer - is applied. To allow for fine-tuning, its 'trainable' parameter is set to 'True'. This part of code, as well as an inspiration for using the BERT model this way, was taken from this great notebook: https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub.

Imbalance of classes (only 747 instances of "spam") is compensated by setting custom class weights for the training loss function.

Once trained, the model can be used for inference, i.e. predicting whether a particular SMS would be classified as "spam" or not. For test purposes, I handcrafted a bunch of messages which I would definitely not want to appear on my phone. A helper function 'check_if_spam' can be used to check for any other message (try it yourself...). Notice that due to very small size of the training sample, model predictions frequently run counterintuitive. More precisely, plenty of 'suspicious' texts are classified as non-spam.

I took an inspiration for this project from the book: 
GULLI, KAPOOR, PAL [2019]: Deep Learning with TensorFlow 2 and Keras - Second Edition, Packt Publishing.

### Setup & Imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
PATH = '/kaggle/input/sms-spam-collection-dataset/'
!ls $PATH

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow_hub as hub

In [ ]:
# importing the official tokenization script created by the Google team
!wget https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

import tokenization

### Data loading & inspection

In [ ]:
sms_data = pd.read_csv(PATH+'spam.csv', encoding='latin_1')
sms_data

In [ ]:
# checking for missing values

sms_data.isna().sum()

In [ ]:
# dropping (almost) empty columns as not important

cols_to_drop = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
sms_data.drop(columns=cols_to_drop, inplace=True)
sms_data

In [ ]:
# renaming feature and target columns
feat_name = 'sms_text'
target_name = 'spam'
sms_data.columns = [target_name, feat_name]

# encoding (binary) target variable
sms_data['target'] = [1 if is_spam == 'spam' else 0 for is_spam in sms_data['spam']]

sms_data

In [ ]:
sms_data.isna().sum()

In [ ]:
# checking (binary) target distribution

sms_data['spam'].value_counts()

In [ ]:
# checking if all sms texts are unique

len(sms_data['sms_text'].unique()) == len(sms_data['sms_text'])

### Model import & data preparation

#### model architecture: 

BERT model available from the TensorFlow Hub

embedding weights are fine-tuned as a part of model training

SOURCE for code and inspiration: https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub; sincere thanks @xhlulu for sharing

##### BERT helper functions to encode texts and build model

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
# setting model optimization parameters:
optimizer = Adam(lr=2e-6)
loss = 'binary_crossentropy'
metrics = ['accuracy']

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

##### Loading BERT from TensorFlow Hub

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)
bert_layer

#### Data tokenization

In [ ]:
# setting the tokenizer parameters
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
# encoding "train" and "test" datasets with the 'bert_encode' helper function

train_input = bert_encode(sms_data['sms_text'].values, tokenizer, max_len=189)

num_seq = train_input[0].shape[0]
len_seq = train_input[0].shape[1]

print('Encoded {} sequences and padded for equal length of {} tokens'
      .format(num_seq, len_seq))

In [ ]:
train_labels = sms_data['target'].values

### Modelling

In [ ]:
model = build_model(bert_layer, max_len=189)
model.summary()

##### setting training parameters

In [ ]:
batch_size = 16
epochs = 10

# setting up the "EarlyStopping" callback
early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0, 
                           patience=3, 
                           verbose=True, 
                           mode='auto', 
                           baseline=None, 
                           restore_best_weights=False)

callbacks = [early_stop]

validation_split = 0.20

# setting class weights for the loss function to adjust for class imbalance
# 'spam' is set to weight 8x more
class_weight = {0: 1, 1: 8}

In [ ]:
# training model with validation and early stopping

model.fit(x=train_input, y=train_labels, 
          batch_size=batch_size, epochs=epochs, 
          verbose=True, callbacks=callbacks, 
          validation_split=validation_split, 
          class_weight=class_weight)

#### Learning history

In [ ]:
# showing history of 'accuracy'

plt.figure()
plt.plot(model.history.history['accuracy'], label='TRAIN ACC')
plt.plot(model.history.history['val_accuracy'], label='VAL ACC')
plt.legend()
plt.show()

In [ ]:
# showing history of 'loss'

plt.figure()
plt.plot(model.history.history['loss'], label='TRAIN LOSS')
plt.plot(model.history.history['val_loss'], label='VAL LOSS')
plt.legend()
plt.show()

#### Model evaluation

In [ ]:
# making predictions for training sequences (in-sample check)

predictions = model.predict(train_input)
predictions.shape

In [ ]:
pred_classes = (predictions > 0.5).astype(int)
pred_classes.shape

In [ ]:
# showing confusion matrix

cm = confusion_matrix(y_true=train_labels, y_pred=pred_classes)
cm = pd.DataFrame(cm)
cm

In [ ]:
# plotting the confusion matrix heatmap

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True)

#### Re-training the model on full train data

In [ ]:
# setting the optimal number of epochs

epochs = early_stop.stopped_epoch + 1

In [ ]:
model.fit(x=train_input, y=train_labels, 
          batch_size=batch_size, epochs=epochs, 
          verbose=True, 
          class_weight=class_weight)

### Predicting

In [ ]:
"""
helper function: check if a SMS text provided would be classified as spam or not
argument: <string> with SMS text to be checked
if no argument provided, read the user's input
"""

def check_if_spam(sms=None):

    # read user's input if no argument provided
    if sms is None:
        sms = input('Enter SMS text: ')
    
    # tokenize the SMS text and pad sequence to match training sequences length
    sms = [sms,]
    sequence = bert_encode(sms, tokenizer, max_len=189)
        
    # predict class and give feedback
    prediction = model.predict(sequence)
    pred_class = (prediction > 0.5).astype(int)
    is_spam = 'This is SPAM !!!' if pred_class == 1 else 'This is not spam.'
        
    return is_spam

In [ ]:
my_sample = ['Final chance to win free tickets. Call now!', 
             'Suspicious activity detected. Follow this link to change password immediately.',
             'Get over here and call me tonite. Only 2 USD for minute.',
             'What are you waiting for! These are final days of our xmass promo deals.',
             'We have new offers for you. Visit our webpage and see.',
             'Binary FX options trading and 100 USD on your account. Hurry up.',
             'Huge discounts this weekend. Check this site to learn more.',
             'You can also earn easy money. Call us now.',
             'Congratulations! to claim your reward you must reply immediately',
             'For our database update we need a contact from you. Call us at.'
            ]

for text in my_sample:
    print('\nChecking:     ', text)
    print(check_if_spam(text))

In [ ]:
# call the 'check_if_spam' function with no arguments to provide custom text
# uncomment to see in action

# check_if_spam()

In [ ]:
# helper script to show random "spam message"

spams = sms_data[sms_data['spam'] == 'spam']
idx = np.random.randint(len(spams))
spam = spams.iloc[idx]['sms_text']
print(spam)